In [ ]:
import pandas as pd
import numpy as np
import glob

In [ ]:
### NSEFUTURES CSV

In [ ]:
path = r'NSEFUTURES/NiftyStocksFutures/' # use your path
all_files = glob.glob(path + "*/*.csv")

li = []
count = 1

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
frame = frame.fillna(value=np.nan)

In [ ]:
frame = frame.fillna(-1)

In [ ]:
list(frame)

In [ ]:
count = 1
ls = []
for i in frame['Symbol'].unique():
    mf = frame.loc[frame['Symbol'] == i]
    mf['stockKey'] = count
    count = count + 1
    ls.append(mf)
    
frame = pd.concat(ls, axis=0, ignore_index=True)

In [ ]:
frame

In [ ]:
frame.rename(columns = {'Open':'Open_Price', 'High':'High_Price', 
                              'Low':'Low_Price', 'Close':'Close_Price', 'LTP':'Last_Price', 'No. of Trades':'No_of_Trades','Deliverable Qty':'Deliverable_Qty','%Dly Qt to Traded Qty':'%Dly_Qt_to_Traded_Qty', 'Settle Price':'Settle_Price', 'No. of contracts': 'No_of_Contracts','Turnover in Lacs':'Turnover_in_Lacs','Open Int':'Open_Int','Change in OI':'Change_in_OI','Underlying Value ':'Underlying_Value'}, inplace = True) 

In [ ]:
frame['Date']

frame['Date'] = pd.to_datetime(frame.Date)

In [ ]:
frame['Date'] = frame['Date'].dt.strftime('%Y-%m-%d')

In [ ]:
frame['Date']

In [ ]:
for i in range(0, len(frame), 20000):
    col1 = frame[i:i+20000]
#     col1 = col1.to_frame()
    col1.to_csv(r"NiftyStocksFutures_col_"+str(i)+".csv")

In [ ]:
frame.to_csv("nsefutures_v2.csv")

In [ ]:
# NSESECURITIES CSV

In [ ]:
path = r'NSESECURITIES/' # use your path
all_files = glob.glob(path + "*/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
frame = frame.fillna(value=np.nan)

In [ ]:
frame = frame.fillna(-1)

In [ ]:
frame.shape

In [ ]:
count = 1
ls = []
for i in frame['Symbol'].unique():
    mf = frame.loc[frame['Symbol'] == i]
    mf['stockKey'] = count
    count = count + 1
    ls.append(mf)
    
frame = pd.concat(ls, axis=0, ignore_index=True)

In [ ]:
frame.rename(columns = {'Open Price':'Open_Price', 'High Price':'High_Price', 
                              'Low Price':'Low_Price', 'Close Price':'Close_Price', 'Last Price':'Last_Price', 'Prev Close':'Prev_Close', 'Average Price':'Average_Price', 'Total Traded Quantity':'Total_Traded_Quantity','No. of Trades':'No_Of_Trades','Deliverable Qty':'Deliverable_Qty','% Dly Qt to Traded Qty':'%Dly_Qt_to_Traded_Qty'}, inplace=True)

In [ ]:
frame['Date']

frame['Date'] = pd.to_datetime(frame.Date)

In [ ]:
frame['Date'] = frame['Date'].dt.strftime('%Y-%m-%d')

In [ ]:
frame.columns

In [ ]:
for i in range(0, len(frame), 20000):
    col1 = frame[i:i+20000]
#     col1 = col1.to_frame()
    col1.to_csv(r"nsesec_col_"+str(i)+".csv")

In [ ]:
frame.to_csv("nsesecurities_v2.csv")

In [ ]:
list(frame)

In [1]:
import pandas as pd
import numpy as np
from py2neo import Graph
from neo4j import GraphDatabase

In [2]:
# df = pd.read_parquet('product_master_05062020230002.parquet/product_master_05062020230002.parquet', engine='pyarrow')


In [3]:
from neo4j import __version__ as neo4j_version
print(neo4j_version)

4.1.1


In [4]:
from neo4j import GraphDatabase

class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [5]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="karan", pwd="root")

In [6]:
#list(frame)

In [7]:
conn.query("CREATE OR REPLACE DATABASE stock")

[]

In [8]:
# ### lOAD DATE NODES

# query_string = '''
# LOAD CSV WITH HEADERS FROM 'file:///nsesec_col_20000.csv' AS row
# MERGE (s:Stock { Symbol: row.Symbol, Series: row.Series, Prev_Close: row.Prev_Close, Open_Price: row.Open_Price, High_Price: row.High_Price, Low_Price:row.Low_Price,Last_Price:row.Last_Price,Close_Price:row.Close_Price,Average_Price:row.Average_Price,Total_Traded_Quantity:row.Total_Traded_Quantity })
# MERGE (d:Date {Date: row.Date})

# '''
# # started at 10 05
# conn.query(query_string, db='stock')

In [9]:
### lOAD DATE NODES

query_string = '''

LOAD CSV WITH HEADERS FROM 'file:///date_1980_onwards.csv' AS line
CREATE (a: Date { date: line.Date})

'''
# started at 10 05
conn.query(query_string, db='stock')

[]

In [10]:
query_string = '''
MATCH (n) RETURN count(n)
'''

conn.query(query_string, db='stock')

[<Record count(n)=14884>]

In [11]:
### LOAD STOCK DATA AND POINT THEM TO THEIR RESPECTIVE DATES

query_string = '''
LOAD CSV WITH HEADERS FROM 'file:///nsesecurities_v2.csv' AS row
CREATE (s:Stock { Symbol: row.Symbol, Series: row.Series, Prev_Close: row.Prev_Close, Open_Price: row.Open_Price, High_Price: row.High_Price, Low_Price:row.Low_Price,Last_Price:row.Last_Price,Close_Price:row.Close_Price,Average_Price:row.Average_Price,Total_Traded_Quantity:row.Total_Traded_Quantity, stockKey:row.stockKey })
MERGE (d:Date {Date: row.Date})

MERGE (s)<-[:HAS_DATE]->(d)
'''
# started at 11 40
conn.query(query_string, db='stock')

[]

In [12]:
query_string = '''
MATCH (n) RETURN count(n)
'''

conn.query(query_string, db='stock')

[<Record count(n)=311685>]

In [13]:

query_string = '''
LOAD CSV WITH HEADERS FROM 'file:///nsefutures_v2.csv' AS row
CREATE (s:Stock { Symbol: row.Symbol,Date: row.Date, Expiry: row.Expiry, Open_Price:row.Open_Price,High_Price:row.High_Price, Low_Price:row.Low_Price, Close_Price:row.Close_Price, Last_Price:row.Last_Price,Settle_Price:row.Settle_Price,No_of_Contracts:row.No_of_Contracts,Turnover_in_Lacs:row.Turnover_in_Lacs, Open_Int:row.Open_Int,Change_in_OI:row.Change_in_OI,Underlying_Value:row.Underlying_Value,stockKey:row.stockKey})
MERGE (d:Date {Date: row.Date})

MERGE (s)<-[:HAS_DATE]->(d)
'''
# started at 11 40
conn.query(query_string, db='stock')

[]

In [14]:
query_string = '''
MATCH (n) RETURN count(n)
'''

conn.query(query_string, db='stock')

[<Record count(n)=837042>]

In [15]:
### LOAD COMPANY NODES AND POINT THEM TO THEIR RESPECTIVE STOCK NODES

query_string = '''
LOAD CSV WITH HEADERS FROM 'file:///Stock_cin_v2.csv' AS row
CREATE (c:Company { Symbol: row.Symbol, Alternate_Name: row.Alternate_Name, CIN: row.CIN, Company_Name: row.Company_Name, Company_Status: row.Company_Status, RoC:row.RoC,Registration_Number:row.Registration_Number,Company_Category:row.Company_Category,Company_Sub_Category:row.Company_Sub_Category,Class_of_Company:row.Class_of_Company, Date_of_Incorporation:row.Date_of_Incorporation,Age_of_Company:row.Age_of_Company,Activity:row.Activity, Number_of_members:row.Number_of_members, Authorised_Capital:row.Authorised_Capital,Paid_up_Capital:row.Paid_up_Capital,Email:row.Email
 })
 



'''
# started at 10 05
conn.query(query_string, db='stock')

[]

In [16]:
query_string = '''
MATCH (n) RETURN count(n)
'''

conn.query(query_string, db='stock')

[<Record count(n)=837095>]

In [17]:
### CREATE RELATIONSHIP BETWEEN COMPANY AND STOCK

query_string = '''

MATCH (c:Company),(s:Stock)
WHERE c.Symbol = s.Symbol
CREATE (c)-[:HAS_STOCK]->(s)

'''
# started at 10 05
conn.query(query_string, db='stock')

[]

In [18]:
query_string = '''
MATCH (n) RETURN count(n)
'''

conn.query(query_string, db='stock')

[<Record count(n)=837095>]

In [19]:
### LOAD DIRECTOR DATA

query_string = '''
LOAD CSV WITH HEADERS FROM 'file:///Directors_v2.csv' AS row
CREATE (d:Director { Symbol: row.Symbol, DIN: row.DIN, Director_Name: row.Director_Name, Designation: row.Designation, Appointment_Date:row.Appointment_Date})
 
'''
# started at 10 05
conn.query(query_string, db='stock')

[]

In [20]:
query_string = '''
MATCH (n) RETURN count(n)
'''

conn.query(query_string, db='stock')

[<Record count(n)=837525>]

In [21]:
### CONNECT DIRECTORS TO COMPANIES

query_string = '''

MATCH (d:Director),(c:Company)
WHERE d.Symbol = c.Symbol
CREATE (d)-[:DIRECTOR_AT]->(c)


'''
# started at 10 05
conn.query(query_string, db='stock')

[]

In [22]:
query_string = '''
MATCH (n) RETURN count(n)
'''

conn.query(query_string, db='stock')

[<Record count(n)=837525>]

In [ ]:
####################################################################################################################################

In [ ]:
query_string = '''
USING PERIODIC COMMIT 10000
LOAD CSV FROM 'file:///nsesec_col_120000.csv' AS line
MERGE(a: Symbol { Symbol: line[1]})
MERGE (b: Series {Series: line[2]})
MERGE(c: Date { Date: line[3]})
MERGE(d: Prev_Close { Prev_Close: line[4]})
MERGE(e: Open { Open: line[5]})
MERGE(f: High { High: line[6]})
MERGE(g: Low { Low: line[7]})
MERGE(h: LTP { LTP: line[8]})
MERGE(i: Close { Close: line[9]})
MERGE(j: Average_Price {Average_Price: line[10]})
MERGE(k: Total_Traded_Quantity {Total_Traded_Quantity : line[11]})
MERGE(l: Turnover {Turnover : line[12]})
MERGE(m: No_of_Trades {No_Of_Trades: line[13]})
MERGE(n: Deliverable_Qty {Deliverable_Qty: line[14]})
MERGE(o: Deliverable_Qty_To_Traded_Qty {Deliverable_Qty_To_Traded_Qty: line[15]})
//Relationships::


MERGE (a)-[:Series]->(b)
MERGE (a)-[:Date]->(c)
MERGE (a)-[:Prev_Close]->(d)
MERGE (a)-[:Open]->(e)
MERGE (a)-[:High]->(f)
MERGE (a)-[:Low]->(g)
MERGE (a)-[:LTP]->(h)
MERGE (a)-[:Close]->(i)
MERGE (a)-[:Average_Price]->(j)
MERGE (a)-[:Total_Traded_Quantity]->(k)
MERGE (a)-[:Turnover]->(l)
MERGE (a)-[:No_of_Trades]->(m)
MERGE (a)-[:Deliverable_Qty]->(n)
MERGE (a)-[:Deliverable_Qty_To_Traded_Qty]->(o)

'''
# started at 10 05
conn.query(query_string, db='testpm')

In [ ]:
query_string = '''
MATCH (n) RETURN count(n)
'''

conn.query(query_string, db='testpm')

In [ ]:
query_string = '''
USING PERIODIC COMMIT 10000
LOAD CSV FROM 'file:///nsesec_col_140000.csv' AS line
MERGE(a: Symbol { Symbol: line[1]})
MERGE (b: Series {Series: line[2]})
MERGE(c: Date { Date: line[3]})
MERGE(d: Prev_Close { Prev_Close: line[4]})
MERGE(e: Open { Open: line[5]})
MERGE(f: High { High: line[6]})
MERGE(g: Low { Low: line[7]})
MERGE(h: LTP { LTP: line[8]})
MERGE(i: Close { Close: line[9]})
MERGE(j: Average_Price {Average_Price: line[10]})
MERGE(k: Total_Traded_Quantity {Total_Traded_Quantity : line[11]})
MERGE(l: Turnover {Turnover : line[12]})
MERGE(m: No_of_Trades {No_Of_Trades: line[13]})
MERGE(n: Deliverable_Qty {Deliverable_Qty: line[14]})
MERGE(o: Deliverable_Qty_To_Traded_Qty {Deliverable_Qty_To_Traded_Qty: line[15]})
//Relationships::


MERGE (a)-[:Series]->(b)
MERGE (a)-[:Date]->(c)
MERGE (a)-[:Prev_Close]->(d)
MERGE (a)-[:Open]->(e)
MERGE (a)-[:High]->(f)
MERGE (a)-[:Low]->(g)
MERGE (a)-[:LTP]->(h)
MERGE (a)-[:Close]->(i)
MERGE (a)-[:Average_Price]->(j)
MERGE (a)-[:Total_Traded_Quantity]->(k)
MERGE (a)-[:Turnover]->(l)
MERGE (a)-[:No_of_Trades]->(m)
MERGE (a)-[:Deliverable_Qty]->(n)
MERGE (a)-[:Deliverable_Qty_To_Traded_Qty]->(o)
'''
# started at 10 05
conn.query(query_string, db='testpm')

In [ ]:
query_string = '''
MATCH (n) RETURN count(n)
'''

conn.query(query_string, db='testpm')

In [ ]:
query_string = '''
USING PERIODIC COMMIT 10000
LOAD CSV FROM 'file:///nsesec_col_160000.csv' AS line
MERGE(a: Symbol { Symbol: line[1]})
MERGE (b: Series {Series: line[2]})
MERGE(c: Date { Date: line[3]})
MERGE(d: Prev_Close { Prev_Close: line[4]})
MERGE(e: Open { Open: line[5]})
MERGE(f: High { High: line[6]})
MERGE(g: Low { Low: line[7]})
MERGE(h: LTP { LTP: line[8]})
MERGE(i: Close { Close: line[9]})
MERGE(j: Average_Price {Average_Price: line[10]})
MERGE(k: Total_Traded_Quantity {Total_Traded_Quantity : line[11]})
MERGE(l: Turnover {Turnover : line[12]})
MERGE(m: No_of_Trades {No_Of_Trades: line[13]})
MERGE(n: Deliverable_Qty {Deliverable_Qty: line[14]})
MERGE(o: Deliverable_Qty_To_Traded_Qty {Deliverable_Qty_To_Traded_Qty: line[15]})
//Relationships::


MERGE (a)-[:Series]->(b)
MERGE (a)-[:Date]->(c)
MERGE (a)-[:Prev_Close]->(d)
MERGE (a)-[:Open]->(e)
MERGE (a)-[:High]->(f)
MERGE (a)-[:Low]->(g)
MERGE (a)-[:LTP]->(h)
MERGE (a)-[:Close]->(i)
MERGE (a)-[:Average_Price]->(j)
MERGE (a)-[:Total_Traded_Quantity]->(k)
MERGE (a)-[:Turnover]->(l)
MERGE (a)-[:No_of_Trades]->(m)
MERGE (a)-[:Deliverable_Qty]->(n)
MERGE (a)-[:Deliverable_Qty_To_Traded_Qty]->(o)
'''
# started at 10 05
conn.query(query_string, db='testpm')

In [ ]:
query_string = '''
MATCH (n) RETURN count(n)
'''

conn.query(query_string, db='testpm')

In [ ]:
query_string = '''
USING PERIODIC COMMIT 10000
LOAD CSV FROM 'file:///nsesec_col_180000.csv' AS line
MERGE(a: Symbol { Symbol: line[1]})
MERGE (b: Series {Series: line[2]})
MERGE(c: Date { Date: line[3]})
MERGE(d: Prev_Close { Prev_Close: line[4]})
MERGE(e: Open { Open: line[5]})
MERGE(f: High { High: line[6]})
MERGE(g: Low { Low: line[7]})
MERGE(h: LTP { LTP: line[8]})
MERGE(i: Close { Close: line[9]})
MERGE(j: Average_Price {Average_Price: line[10]})
MERGE(k: Total_Traded_Quantity {Total_Traded_Quantity : line[11]})
MERGE(l: Turnover {Turnover : line[12]})
MERGE(m: No_of_Trades {No_Of_Trades: line[13]})
MERGE(n: Deliverable_Qty {Deliverable_Qty: line[14]})
MERGE(o: Deliverable_Qty_To_Traded_Qty {Deliverable_Qty_To_Traded_Qty: line[15]})
//Relationships::


MERGE (a)-[:Series]->(b)
MERGE (a)-[:Date]->(c)
MERGE (a)-[:Prev_Close]->(d)
MERGE (a)-[:Open]->(e)
MERGE (a)-[:High]->(f)
MERGE (a)-[:Low]->(g)
MERGE (a)-[:LTP]->(h)
MERGE (a)-[:Close]->(i)
MERGE (a)-[:Average_Price]->(j)
MERGE (a)-[:Total_Traded_Quantity]->(k)
MERGE (a)-[:Turnover]->(l)
MERGE (a)-[:No_of_Trades]->(m)
MERGE (a)-[:Deliverable_Qty]->(n)
MERGE (a)-[:Deliverable_Qty_To_Traded_Qty]->(o)

'''
# started at 10 05
conn.query(query_string, db='testpm')

In [ ]:
query_string = '''
MATCH (n) RETURN count(n)
'''

conn.query(query_string, db='testpm')

In [ ]:
query_string = '''
USING PERIODIC COMMIT 10000
LOAD CSV FROM 'file:///nsesec_col_200000.csv' AS line
MERGE(a: Symbol { Symbol: line[1]})
MERGE (b: Series {Series: line[2]})
MERGE(c: Date { Date: line[3]})
MERGE(d: Prev_Close { Prev_Close: line[4]})
MERGE(e: Open { Open: line[5]})
MERGE(f: High { High: line[6]})
MERGE(g: Low { Low: line[7]})
MERGE(h: LTP { LTP: line[8]})
MERGE(i: Close { Close: line[9]})
MERGE(j: Average_Price {Average_Price: line[10]})
MERGE(k: Total_Traded_Quantity {Total_Traded_Quantity : line[11]})
MERGE(l: Turnover {Turnover : line[12]})
MERGE(m: No_of_Trades {No_Of_Trades: line[13]})
MERGE(n: Deliverable_Qty {Deliverable_Qty: line[14]})
MERGE(o: Deliverable_Qty_To_Traded_Qty {Deliverable_Qty_To_Traded_Qty: line[15]})
//Relationships::


MERGE (a)-[:Series]->(b)
MERGE (a)-[:Date]->(c)
MERGE (a)-[:Prev_Close]->(d)
MERGE (a)-[:Open]->(e)
MERGE (a)-[:High]->(f)
MERGE (a)-[:Low]->(g)
MERGE (a)-[:LTP]->(h)
MERGE (a)-[:Close]->(i)
MERGE (a)-[:Average_Price]->(j)
MERGE (a)-[:Total_Traded_Quantity]->(k)
MERGE (a)-[:Turnover]->(l)
MERGE (a)-[:No_of_Trades]->(m)
MERGE (a)-[:Deliverable_Qty]->(n)
MERGE (a)-[:Deliverable_Qty_To_Traded_Qty]->(o)

'''
# started at 10 05
conn.query(query_string, db='testpm')

In [ ]:
query_string = '''
MATCH (n) RETURN count(n)
'''

conn.query(query_string, db='testpm')